In [1]:
#NNJM
#TODO
# 1. Check to be sure slices are being done correctly
# 2. Put in dropout
# 3. 

In [2]:
#Import statements
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#Parameters for the model
params = {}
params['minibatch'] = 50
params['emb-size'] = 200
params['hiddenstate-size'] = 512
params['source-vocab'] = 500
params['target-vocab'] = 600
params['datatype'] = tf.float32
params['init-method'] = 'uniform'
params['init-range'] = 0.01
params['source-window'] = 11
params['target-window'] = 4
params['use-char'] = False
params['seed'] = 1
params['dropout-rate'] = 0.0
params['loss'] = 'MLE' #('MLE','NCE','IS')
params['learning-rate'] = 0.5

In [4]:
#Interactive session
sess = tf.InteractiveSession()

In [5]:
#Process the data, get the source-vocab and target-vocab variables


In [6]:
#Place holders for input and output data, first index is the minibatch size

#For the input the second dimension will be passed a
#     vector of size minibatch x (params['source-window']+params['target-window'])
input_indices = tf.placeholder(tf.int32, shape=[None, params['source-window']+params['target-window']])

#The output will be given a minibatch vector of correct indicies
correct_output = tf.placeholder(tf.int32,shape=[None])

In [7]:
def param_init(params,shape=None,name=None,datatype=None):
    if datatype == None:
        datatype = params['datatype']
    assert shape != None,"Error shape cannot be None in param_init"
    if params['init-method'] == 'uniform':
        return tf.random_uniform(shape, minval=-1*params['init-range'],maxval=params['init-range'],\
                    dtype=datatype, seed=params['seed'], name=name)
    else:
        print "ERROR this init-method has not been created yet"

In [20]:
#Parameters for the source and target embeddings
source_emb_matrix = tf.Variable(param_init(params,shape=[params['source-vocab'],params['emb-size']],name='src-emb'))
target_emb_matrix = tf.Variable(param_init(params,shape=[params['target-vocab'],params['emb-size']],name='tgt-emb'))

In [9]:
#Do the one-hot emebedding lookups
src_embed = tf.nn.embedding_lookup(source_emb_matrix, tf.slice(input_indices,[0,0],[-1,params['source-window']]))
tgt_embed = tf.nn.embedding_lookup(target_emb_matrix, tf.slice(input_indices,[0,params['source-window']],\
                                                                [-1,params['target-window']]))

In [10]:
#Now reshape to be able to feed through non-linearity
print src_embed.get_shape
print tgt_embed.get_shape
concat_embed = tf.concat(1, [src_embed, tgt_embed])
print concat_embed
concat_embed = tf.reshape(concat_embed,[-1,params['emb-size']*(params['source-window']+params['target-window'])])
print concat_embed

<bound method Tensor.get_shape of <tf.Tensor 'embedding_lookup:0' shape=(?, 11, 200) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'embedding_lookup_1:0' shape=(?, 4, 200) dtype=float32>>
Tensor("concat:0", shape=(?, 15, 200), dtype=float32)
Tensor("Reshape:0", shape=(?, 3000), dtype=float32)


In [26]:
#First Layer
layer_1_weights = tf.Variable(param_init(params,\
    shape=[params['emb-size']*(params['source-window']+params['target-window']),params['hiddenstate-size']],name='lyr-1'))
layer_1_bias = tf.Variable(param_init(params,shape=[params['hiddenstate-size']],name='lyr-1-bias'))
layer_1_output = tf.nn.relu(tf.matmul(concat_embed,layer_1_weights)+layer_1_bias)
print layer_1_output.get_shape

<bound method Tensor.get_shape of <tf.Tensor 'Relu_1:0' shape=(?, 512) dtype=float32>>


In [27]:
#Second layer
layer_2_weights = tf.Variable(param_init(params,shape=[params['hiddenstate-size'],params['hiddenstate-size']],\
                        name='lyr-2'))
layer_2_bias = tf.Variable(param_init(params,shape=[params['hiddenstate-size']],name='lyr-2-bias'))
layer_2_output = tf.nn.relu(tf.matmul(layer_1_output,layer_2_weights)+layer_2_bias)

In [32]:
#Softmax layer
softmax_weights = tf.Variable(param_init(params,shape=[params['hiddenstate-size'],params['target-vocab']],\
                              name='softmax-weights')) 
softmax_bias = tf.Variable(param_init(params,shape=[params['target-vocab']],name='softmax-bias'))
final_output = tf.nn.softmax(tf.matmul(layer_2_output, softmax_weights) + softmax_bias)
loss = #use embedding lookup to 
tf.nn.embedding_lookup(, tf.slice(input_indices,[0,0],[-1,params['source-window']]))

<bound method Tensor.get_shape of <tf.Tensor 'Softmax_1:0' shape=(?, 600) dtype=float32>>
